# Weaker and stronger stable set formulations for a scheduling problem

<b>Goal:</b> Implement and compare two stable set IP formulations for a scheduling problem.

## Implementation 

We provide a framework for your implementation below. Your task is to implement the two functions `get_canonical_formulation()` and `get_strengthened_formulation()` inside the class `SchedulingInstance`.

In [1]:
from itertools import chain, combinations
from mip import *
import random
from time import process_time

In [2]:
def interval_overlaps(I1, I2):
    '''Determines whether interval I1 overlaps with I2.'''
    return max(I1[0], I2[0]) < min(I1[1], I2[1])


def interval_contains(I, t):
    '''Determines whether interval I contains point t.'''
    return I[0] <= t < I[1]


class SchedulingInstance:
    def __init__(self, k, p, I, q, T):
        '''Initializes an instance of the scheduling problem. Tasks are represented by the integer indices 0, ..., k-1.
        
        Args:
            k: Number of available tasks.
            p: List of task profits, where profit at index i is the profit of task i.
            I: List of pairs of endpoints of time intervals, where the time interval at index i is the interval of task i.
            q: Number of task groups.
            T: List of sets of tasks describing task groups.
        
        Members:
            group: A dict containing group number for every task.
        '''
        self.k, self.p, self.I, self.q, self.T = k, p, I, q, T
        self.group = {i: l for l in range(q) for i in T[l]}

    def get_canonical_formulation(self):
        '''Constructs a canonical stable set IP formulation of the scheduling problem.

        Returns:
            A dict of python-mip variables and a python-mip model as a tuple.
        '''
        
        # Write your implementation here
        
        model = Model('Canonical stable set IP formulation', sense=MAXIMIZE)

        # variables
        x = {i : model.add_var(name=f"x_{i}", var_type=BINARY) for i in range(self.k)}
        
        # objective
        model += xsum([self.p[i] * x[i] for i in range(self.k)])

        # constraints (one per edge)
        for i, j in combinations(range(self.k), 2):
            if interval_overlaps(self.I[i], self.I[j]) or self.group[i] == self.group[j]:
                model += x[i] + x[j] <= 1

        return (x, model)
    
    def get_strengthened_formulation(self):
        '''Constructs a strengthened IP of the scheduling problem.

        Returns:
            A dict of python-mip variables and a python-mip model as a tuple.
        '''
        
        # Write your implementation here
        
        model = Model('Strengthened stable set IP formulation', sense=MAXIMIZE)
        # variables
        x = {i : model.add_var(name=f"x_{i}", var_type=BINARY) for i in range(self.k)}
        
        # objective
        model += xsum([self.p[i] * x[i] for i in range(self.k)])

        # constraints of type 1
        for l in range(self.q):
            model += xsum([x[i] for i in self.T[l]]) <= 1

        # constraints of type 2 
        for i, j in combinations(range(self.k), 2):
            if interval_overlaps(self.I[i], self.I[j]):
                model += x[i] + x[j] <= 1

        return (x, model)
    
    def get_tasks_from_var(self, x):
        '''Extracts a set of scheduled tasks from an IP variable x.'''
        return {i for i in x if x[i].x >= 0.99}

    def test_schedule(self, tasks):
        '''Tests if a set of tasks forms a feasible schedule.'''
        for i, j in combinations(tasks, 2):
            if interval_overlaps(self.I[i], self.I[j]) or self.group[i] == self.group[j]:
                return False
        return True

    def test_formulation(self, formulation, expected_value=None):
        '''Tests if canonical formulation gives a correct schedule and measures its performance.
        
        Args:
            formulation: String 'canonical' or 'strengthened' for which formulation to test.
            expected_value: Expected profit of an optimal schedule.
        
        Returns:
            True on success, False on failure.
        '''
        form_dict = {'canonical': self.get_canonical_formulation, 'strengthened': self.get_strengthened_formulation}
        
        if formulation not in form_dict:
            print(f'Error: unknown formulation {formulation}.')
            return
        
        x, model = form_dict[formulation]()
        
        start = process_time()
        status = model.optimize()
        end = process_time()
        
        print(f'Info: optimization of {formulation} IP elapsed {end - start} seconds.')
        
        if status != OptimizationStatus.OPTIMAL:
            print(f'Error: failed to solve {formulation} IP to optimality. Optimization status: {status}.')
            return
        
        schedule = self.get_tasks_from_var(x)
        res = self.test_schedule(schedule)
        
        if not res:
            print(f'Error: {formulation} IP gave an infeasible schedule {schedule}.')
            return
        
        if expected_value is None:
            print(f'Info: {formulation} IP gave a schedule of value {model.objective_value}.')
        elif abs(model.objective_value - expected_value) <= 1e-6:
            print(f'Info: {formulation} IP gave a schedule of value {model.objective_value}, as expected.')
        else:
            print(f'Error: {formulation} IP gave a schedule of value {model.objective_value} instead of {expected_value}.')
            return

## Testing


In [4]:
# Instance 1
instance1 = SchedulingInstance(4, [1, 1, 1, 1], [(0, 1), (0, 1), (0, 1), (1, 2)], 2, [{0}, {1, 2, 3}])
instance1.test_formulation('canonical', 2)
instance1.test_formulation('strengthened', 2)

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 5 (0) rows, 4 (0) columns and 10 (0) elements
Clp1000I sum of infeasibilities 0 - average 0, 0 fixed columns
Coin0506I Presolve 5 (0) rows, 4 (0) columns and 10 (0) elements
Clp0006I 0  Obj 2 Dual inf 400 (4)
Clp0029I End of values pass after 4 iterations
Clp0000I Optimal - objective value 2
Clp0000I Optimal - objective value 2
Clp0000I Optimal - objective value 2
Clp0032I Optimal objective 2 - 0 iterations time 0.002, Idiot 0.00

Starting MIP optimization
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 

In [5]:
# Instance 2
instance2 = SchedulingInstance(4, [1, 1, 1, 1], [(0, 2), (1, 3), (1, 3), (2, 4)], 2, [{0, 3}, {1, 2}])
instance2.test_formulation('canonical', 1)
instance2.test_formulation('strengthened', 1)

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 6 (0) rows, 4 (0) columns and 12 (0) elements
Clp1000I sum of infeasibilities 1.1724e-12 - average 1.95399e-13, 0 fixed columns
Coin0506I Presolve 6 (0) rows, 4 (0) columns and 12 (0) elements
Clp0006I 0  Obj 2 Dual inf 400 (4)
Clp0029I End of values pass after 4 iterations
Clp0000I Optimal - objective value 2
Clp0000I Optimal - objective value 2
Clp0000I Optimal - objective value 2
Clp0032I Optimal objective 2 - 0 iterations time 0.002, Idiot 0.00

Starting MIP optimization
Cgl0003I 0 fixed, 0 tightened bounds, 3 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 3 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 1 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Streng

For the random instances below, we provide the expected values for 10 instances. As always, you can also choose different seeds beyond 0, ..., 9 to test your implementation!

In [6]:
# Instance 3 - random instances

expected_value = [
    44.05245303197996, 26.242931305742747, 11.384576285274106, 33.19961286186241, 32.880978975068444, 
    52.72533015087802, 52.72091393106317, 41.33415696704504, 29.742720594991482, 49.209142724214495
]

for seed in range(10):
    print(f"Starting random instance {seed}.")
    random.seed(seed)

    k = 200
    q = random.randint(1, k)
    p = [random.random() for i in range(k)]
    I = [(random.random(), random.random()) for i in range(k)]
    T = list(range(k))
    random.shuffle(T)
    split_points = sorted(random.sample(range(k), q - 1))
    sublists = zip(chain([0], split_points), chain(split_points, [None]))
    T = list(T[i: j] for i, j in sublists)

    instance3 = SchedulingInstance(k, p, I, q, T)
    instance3.test_formulation('canonical', expected_value[seed])
    instance3.test_formulation('strengthened', expected_value[seed])
    print("---------------------------------")

Starting random instance 0.
Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 3247 (-554) rows, 141 (-59) columns and 6494 (-1108) elements
Clp1000I sum of infeasibilities 7.78839e-07 - average 2.39864e-10, 0 fixed columns
Coin0506I Presolve 3247 (0) rows, 141 (0) columns and 6494 (0) elements
Clp0006I 0  Obj 59.850371 Dual inf 7090.3858 (141)
Clp0029I End of values pass after 141 iterations
Clp0014I Perturbing problem by 0.001% of 1 - largest nonzero change 2.9995211e-05 ( 0.0014997605%) - largest zero change 2.9943246e-05
Clp0000I Optimal - objective value 59.926502
Clp0000I Optimal - objective value 59.926502
Clp0000I Optimal - objective value 59.926502
Coin0511I After Postsolve, objective 59.926502, infeasibilities - dual 1.8475342 (6), primal 0 (0)
Coin0512I Presolved model was optimal, full model needs cleaning up
Clp0006I 0  Obj 59.926502 Dual inf 1.8475288 (6)
Clp0000I Optimal - objective value 59.926502
Clp0032I Optimal obj